1. PDF → Metin al ve node çıkar (llm ile otomatik) nodeları etiketle[[]] ile

In [1]:
text = "Yapay zeka sistemlerinin etik yönleri çok önemlidir. Regülasyon konusu da bununla ilgilidir."
concepts = ["Yapay zeka", "etik", "Regülasyon"]
import re

def link_concepts(text, concepts):
    for concept in sorted(concepts, key=len, reverse=True):
        text = re.sub(rf"\b({re.escape(concept)})\b", r"[[\1]]", text)
    return text

linked_text = link_concepts(text, concepts)
print(linked_text)


[[Yapay zeka]] sistemlerinin [[etik]] yönleri çok önemlidir. [[Regülasyon]] konusu da bununla ilgilidir.


 Node ve Edge çıkart + bağlamları tut

In [2]:
def extract_edges_and_contexts(linked_text, main_node="Text_1"):
    pattern = r'(.{0,50})\[\[(.*?)\]\](.{0,50})'
    matches = re.findall(pattern, linked_text)

    edges, contexts, nodes = [], [], set()
    for before, concept, after in matches:
        context = f"{before.strip()} {concept} {after.strip()}"
        edges.append({"from": main_node, "to": concept})
        contexts.append(context)
        nodes.add(concept) 

    nodes.add(main_node)
    return list(nodes), edges, contexts
nodes, edges, contexts = extract_edges_and_contexts(linked_text)

TF-IDF ile edge etiketleri (bağlam kelimesi) ya da llm e etik’ arasında nasıl bir ilişki vardır? Bunu bir kelime veya kısa bir açıklama ile özetle.”

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

def label_edges_with_tfidf(contexts):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(contexts)
    labels = []
    for i in range(len(contexts)):
        row = X[i].toarray()[0]
        top_index = row.argmax()
        labels.append(vectorizer.get_feature_names_out()[top_index])
    return labels

labels = label_edges_with_tfidf(contexts)


graph.json

In [4]:
def build_graph_json(nodes, edges, labels):
    for i, label in enumerate(labels):
        edges[i]["label"] = label

    node_objs = [{"id": n, "label": n} for n in nodes]
    return {
        "nodes": node_objs,
        "edges": edges
    }

graph = build_graph_json(nodes, edges, labels)

import json
with open("graph.json", "w", encoding="utf-8") as f:
    json.dump(graph, f, ensure_ascii=False, indent=2)


In [5]:
print(json.dumps(graph, indent=2, ensure_ascii=False))


{
  "nodes": [
    {
      "id": "etik",
      "label": "etik"
    },
    {
      "id": "Text_1",
      "label": "Text_1"
    }
  ],
  "edges": [
    {
      "from": "Text_1",
      "to": "etik",
      "label": "da"
    }
  ]
}


Cytoscape.js ile Graph Görselleştirme

In [ ]:
!pip install pyvis


In [6]:
from pyvis.network import Network

# Mini graph oluştur
net = Network(notebook=True, height="400px", width="80%")

# Node'ları ekle
for node in nodes:
    net.add_node(node, label=node)

# Edge'leri ekle
for i, edge in enumerate(edges):
    label = labels[i] if i < len(labels) else ""
    net.add_edge(edge['from'], edge['to'], label=label)

# Görselleştir
net.show("pyvis_graph.html")


pyvis_graph.html


In [7]:
from IPython.display import IFrame
IFrame("pyvis_graph.html", width="100%", height="450px")
